In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from utils import utils, postprocessing_utils as post

In [25]:
def get_scores(group, short_model_name, task, experiment, metric):
    all_groups = ["Fusional", "Isolating", "Agglutinative", "Introflexive"]
    results_path = "../results/{}/{}/results_{}_postprocessed.xlsx".format(experiment, short_model_name, task)
    
    # Clean dataframe to get actual info
    df = post.retrieve_results(results_path)[metric]["groupvgroup"]
    df = df.loc[:3, ["Test\Train", *all_groups]]
    
    other_groups = list(filter(lambda x: x != group, all_groups))
    scores = df[group].values.tolist() + df.loc[df["Test\Train"] == group, other_groups].values.flatten().tolist()
    return scores

In [42]:
groups = ["Isolating", "Introflexive"]
tasks = ["pos", "sentiment"]
metrics = ["Accuracy", "Macro_F1"]
models = ["mbert", "xlm-roberta"]

for g in groups:
    output = "\\{}{{{}}}".format(g.lower(), g)
    for task, metric in zip(tasks, metrics):
        for model in models:
            scores_old = get_scores(g, model, task, "acl", metric)
            scores_new = get_scores(g, model, task, "acl-pretrain", metric)
            avg = np.mean(scores_new) - np.mean(scores_old)
            output += " & {:.1f}".format(avg * 100)
    print(output)

\isolating{Isolating} & 2.9 & 3.5 & 3.3 & 5.7
\introflexive{Introflexive} & 10.6 & 12.4 & 8.9 & 11.9
